In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
import warnings
warnings.filterwarnings("ignore")
from src import scraping as sc
import pandas as pd

## Twitch API get stream list

In [ ]:
# Call the function to get the OAuth token
twitch = sc.TwApi()

In [ ]:
# Call the function to get list of live channels
stream_list = twitch.get_stream_list(n_pages = 1600)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
df = pd.DataFrame(stream_list)[['user_id','user_name', 'game_id', 'language', 'viewer_count', 'game_name', 'type']]
df.head()

In [ ]:
evening11_30_2020 = df.drop_duplicates(
    subset = 'user_id').to_csv(
    'twam12-1-2020.csv', index = False)

## Get user info on tw api

In [ ]:
from glob import glob
big_df = pd.DataFrame()
for f in glob('tw*.csv'):
    big_df = pd.concat([big_df, pd.read_csv(f)])
big_df.drop_duplicates(subset = 'user_id', inplace = True)

In [ ]:
big_df.info()
big_df.head()

In [ ]:
api_list_by_user_id = [pd.DataFrame(
    twitch.request_it('users',
                      {'id': big_df.user_id[i*100:i*100+100]})['data'])
        for i in range(int(len(big_df)/100))]

In [ ]:
df_api_2 = pd.concat(api_list_by_user_id, ignore_index = True)
df_api_2.rename(columns = {'id': 'user_id'}, inplace = True)
df_api_2.user_id = df_api_2.user_id.astype('int64')

In [ ]:
merged_df = big_df.join(df_api_2.set_index('user_id'),
                        on = 'user_id',
                        lsuffix = '_left',
                        rsuffix = '_right')
merged_df.reset_index(drop=True, inplace = True)
merged_df.user_name = merged_df.user_name.str.lower()

In [ ]:
merged_df.head()

In [ ]:
# dropping channels with less than 900 views (75 viewers per 12 streams)
merged_df = merged_df.loc[merged_df.view_count > 900]

In [ ]:
# change string to datetime object in created at column
merged_df.created_at = pd.to_datetime(merged_df.created_at)

In [ ]:
# using the created at datetime column to create an account age timedelta column
merged_df['account_age'] = pd.to_datetime("now", utc = True) - merged_df.created_at
# filtering the accounts that are younger than 60 days old
merged_df = merged_df.loc[merged_df.account_age > pd.to_timedelta('60 days')]

In [ ]:
df_api_2.broadcaster_type.value_counts()

In [ ]:
merged_df.broadcaster_type.value_counts()

In [ ]:
merged_df.reset_index(drop = True, inplace = True)

In [ ]:
merged_df.info()

In [ ]:
merged_df.to_csv('merged_df.csv', index = False)

## Twitchtracker scraping

In [ ]:
import pandas as pd
merged_df = pd.read_csv('merged_df.csv')

In [ ]:
n_samples = int(len(merged_df) / 10)
sample_df = merged_df.sample(n = n_samples, random_state = 100)

In [ ]:
sample_df

In [ ]:
# proxies = sc.get_proxies()
# proxie_pool = cycle(proxies)

In [ ]:
# proxy = next(proxie_pool)
# proxy

In [ ]:
sample_df = pd.read_csv('sample_data.csv')

In [ ]:
sc.scrape(sample_df.login[0:5000])

In [ ]:
sc.scrape(sample_df.login[29000:29644])

In [ ]:
sample_df = sample_df.join(scrape_df.set_index('login'),
               on = 'login',
               lsuffix = '_left',
               rsuffix = '_right').head()

In [ ]:
sample_df.to_csv('sample_data.csv', index=False)

In [ ]:
sample_df.loc[sample_df.login == 'davils9']

In [ ]:
sample_df.login[5000:].head(10)

In [ ]:
103.143.46.27:80
89.36.195.238:35328

In [ ]:
sample_df.broadcaster_type.isna().sum()

In [ ]:
sample_df

In [ ]:
# second_sampling_set = merged_df[~merged_df.index.isin(sample_df.index)]
# n_samples = int(len(second_sampling_set) / 10)
# sample_df_2 = second_sampling_set.sample(n = n_samples, random_state = 100)
# sample_df_2.drop(columns = 'Unnamed: 0', inplace = True)
# sample_df_2.to_csv('second_random_sample.csv', index=False)

In [2]:
sample_df_2 = pd.read_csv('second_random_sample.csv')

In [ ]:
sample_df_2.login

In [ ]:
sample_df_2.login[299:]

In [ ]:
sc.scrape(sample_df_2.login[10000:20000])

Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Success: 0 ozpax 103.1.93.184:55443
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Proxy blocked-retrying
Captcha needed-retrying
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Proxy blocked-retrying
Failed
Success: 1 kaswatka 37.79.254.152:3128
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Proxy blocked-retrying
Failed
Failed
Failed
Failed
Failed
Captcha needed-retrying
Failed
Failed
Failed
Failed
Failed
Fai

In [ ]:
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import requests
ua = UserAgent()

In [ ]:
user = sample_df_2.login[1]
url_base = 'https://twitchtracker.com/'
proxy = '193.36.61.203:8000'
real_user_agent = ua.random
headers = {'User-Agent': real_user_agent}
temp_soup = BeautifulSoup(
    requests.get(
        url_base + user + '/statistics',
        headers=headers,
        proxies={"http": proxy, "https": proxy}
    ).text, 
    'html.parser')

In [4]:
sample_df_2[999:]

,user_id,user_name,game_id,language,viewer_count,game_name,type_left,login,display_name,type_right,broadcaster_type,description,profile_image_url,offline_image_url,view_count,created_at,account_age
999,189659184,luvinthebadass,512709.0,en,3,Call of Duty: Black Ops Cold War,live,luvinthebadass,LuvinTheBadass,NaN,affiliate,🏴‍☠️Badass🇨🇦Amateur Streamer•Twitch Affiliated...,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,8986.0,2018-01-09 02:17:55.142363+00:00,1057 days 12:31:47.355502000
1000,105206927,raring,512709.0,en,2,Call of Duty: Black Ops Cold War,live,raring,Raring,NaN,affiliate,I stream fortnite sometimes. Keyboard and Mous...,https://static-cdn.jtvnw.net/jtv_user_pictures...,NaN,1111.0,2015-10-25 22:08:06.612276+00:00,1863 days 16:41:35.885589000
1001,37446841,aivanabg,32982.0,bg,78,Grand Theft Auto V,live,aivanabg,AivanaBG,NaN,partner,AivanaBG is the first female Partnered streame...,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,4255674.0,2012-11-04 19:45:53.839897+00:00,2948 days 19:03:48.657968000
1002,543911019,larazona_,27471.0,pt,1,Minecraft,live,larazona_,Larazona_,NaN,NaN,Seja bem-vindo ao meu mundo!!!!!! Faço lives d...,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,1140.0,2020-06-14 22:58:50.294898+00:00,169 days 15:50:52.202967000
1003,113220548,msblueshines,512710.0,en,5,Call of Duty: Warzone,live,msblueshines,MsBlueShines,NaN,affiliate,I just want to do this full time.,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,80888.0,2016-01-19 00:30:25.616694+00:00,1778 days 14:19:16.881171000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26674,502963273,roadhog_lover,31376.0,de,5,Terraria,live,roadhog_lover,Roadhog_Lover,NaN,NaN,NaN,https://static-cdn.jtvnw.net/jtv_user_pictures...,NaN,1435.0,2020-03-23 22:57:42.078717+00:00,252 days 15:52:00.419148000
26675,71526179,lleilexe,518184.0,en,7,Phasmophobia,live,lleilexe,lleilEXE,NaN,affiliate,exe!,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,14726.0,2014-09-19 16:23:08.218548+00:00,2264 days 22:26:34.279317000
26676,161837100,el_cabra,21779.0,es,1,League of Legends,live,el_cabra,el_cabra,NaN,affiliate,NaN,https://static-cdn.jtvnw.net/jtv_user_pictures...,NaN,6631.0,2017-06-28 06:35:55.428649+00:00,1252 days 08:13:47.069216000
26677,147444751,tveehead,518184.0,en,2,Phasmophobia,live,tveehead,TVeeHead,NaN,affiliate,Variety streamer with a heavy slant towards ho...,https://static-cdn.jtvnw.net/jtv_user_pictures...,NaN,1176.0,2017-02-10 20:19:24.956224+00:00,1389 days 18:30:17.541641000


In [ ]:
208.80.28.208:8080
167.250.180.86:8080
110.234.22.38:8080